Udacity Data Engineering Nanodegree

# Project 2 : Data Modeling with Cassandra
  
About ETL and event data modeling to create a non-relational database and ETL pipeline for a music streaming application. We define queries and tables for a database built using Apache Cassandra.   

---

# Part I. ETL Pipeline

for Pre-Processing the Files

#### Import Python packages 

In [3]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking the current working directory
#print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
#print('My path :')


# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# print(len(full_data_rows_list))

In [6]:
#print(full_data_rows_list)

# creating a smaller event data csv file that be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [7]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


___

# Part II. Apache Cassandra 

#### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear:  
<img src="image/image_event_datafile_new.jpg">

#### Creating a Cluster

In [8]:
# This should make a connection to a Cassandra instance the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [9]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION = 
{'class' : 'SimpleStrategy', 'replication_factor' : 1}
""")

#### Set Keyspace

In [10]:
session.set_keyspace('udacity')

### Create queries to ask the following three questions of the data

#### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




---

## Query 1

![quer1](./image/quer1.png)  
**Query**: Give the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4.  

**Reflexion**: As we search in `sessionId` and `itemInSession`,  they become the primary key. `itemInSession` as clustering key, create logical sets
`artist`, `song` and `length` are what we looking for. 

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_by_session"
query = query + "(sessionId INT, itemInSession INT, artist VARCHAR, song VARCHAR, length DECIMAL, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                   

![Table](./image/query1_table.png)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_by_session(sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [13]:
## SELECT statement to verify the data was entered into the table as DataFrame
query = "SELECT artist, song, length FROM song_by_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)
    
df[['artist', 'song', 'length']].style.hide_index()

artist,song,length
Faithless,Music Matters (Mark Knight Dub),495.3073


## Query 2
![query2](./image/query2.png)
**Query**: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

**Reflexion**: As we search in `userId` and `sessionId`, they becomes the partition key and `itemInSession` the clustering key for sorted the `song`. `artist`, `song`, `fisrtName` and `lastName` are what we looking for. 

In [14]:
query = "CREATE TABLE IF NOT EXISTS song_by_user"
query = query +"(userId INT, sessionId INT, itemInSession INT, artist VARCHAR, song VARCHAR, firstName VARCHAR, lastName VARCHAR, PRIMARY KEY((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

![Table](./image/query2_table.png)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_by_user(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [16]:
# to check the order
query1 = "SELECT userId, sessionId, itemInSession, song FROM song_by_user WHERE userId = 10 AND sessionId = 182"
try:
    df = pd.DataFrame(list(session.execute(query1)))
except Exception as e:
    print(e)

df.style.hide_index()

userid,sessionid,iteminsession,song
10,182,0,Keep On Keepin' On
10,182,1,Greece 2000
10,182,2,Kilometer
10,182,3,Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


In [17]:
query2 = "SELECT artist, song, firstName, lastName FROM song_by_user WHERE userId = 10 AND sessionId = 182"
try:
    df = pd.DataFrame(list(session.execute(query2)))
except Exception as e:
    print(e)
    
df[['artist', 'song', 'firstname', 'lastname']].style.hide_index()


artist,song,firstname,lastname
Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
Three Drives,Greece 2000,Sylvie,Cruz
Sebastien Tellier,Kilometer,Sylvie,Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie,Cruz


## Query 3
![query3](./image/query3.png)  
**Query**: Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'  

**Reflexion**: As we search in `song` it's become the partition key and `userId` the clustering key to find `firstName` and `lastName`for a song.

In [18]:
query = "CREATE TABLE IF NOT EXISTS user_by_song"
query = query + "(song VARCHAR, userId INT, firstName VARCHAR, lastNAME VARCHAR, PRIMARY KEY(song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

![Table](./image/query3_table.png)

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign INSERT statements into the `query` variable
        query = "INSERT INTO user_by_song(song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [20]:
query3 = "SELECT firstName, lastName FROM user_by_song WHERE song='All Hands Against His Own'"
try:
    df = pd.DataFrame(list(session.execute(query3)))
except Exception as e:
    print(e)

df[['firstname', 'lastname']].style.hide_index()

firstname,lastname
Jacqueline,Lynch
Tegan,Levine
Sara,Johnson


---

### Drop the tables before closing out the sessions

In [21]:
query1 = "DROP TABLE song_by_session"
query2 = "DROP TABLE song_by_user"
query3 = "DROP TABLE user_by_song"

try:
    rows1 = session.execute(query1)
    rows2 = session.execute(query2)
    rows3 = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()